## Magentic One

![magentic](https://www.microsoft.com/en-us/research/uploads/prod/2024/11/magnetic_example-2048x1155.png)

In [1]:
%pip install markitdown playwright


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### `magentic_one.py` core code

```python
def __init__(self, client: ChatCompletionClient, hil_mode: bool = False):
    self.client = client
    self._validate_client_capabilities(client)

    fs = FileSurfer("FileSurfer", model_client=client)
    ws = MultimodalWebSurfer("WebSurfer", model_client=client)
    coder = MagenticOneCoderAgent("Coder", model_client=client)
    executor = CodeExecutorAgent("Executor", code_executor=LocalCommandLineCodeExecutor())
    agents: List[ChatAgent] = [fs, ws, coder, executor]
    if hil_mode:
        user_proxy = UserProxyAgent("User")
        agents.append(user_proxy)
    super().__init__(agents, model_client=client)

```

In [2]:
# Autonomously complete a coding task:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.teams.magentic_one import MagenticOne
from autogen_agentchat.ui import Console

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o",
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key
)



client = azure_openai_chat_completion_client
m1 = MagenticOne(client=client)
task = "마이크로소프트 현재 주가와 전망에 대해 알려줘"
result = await Console(m1.run_stream(task=task))
print(result)

/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/var/folders/pr/hngks2mn74d25jycvx6rxl400000gn/T/ipykernel_71380/2988138937.py:25: DeprecationWarning: Instantiating MagenticOne without a code_executor is deprecated. Provide a code_executor to clear this warning (e.g., code_executor=LocalCommandLineCodeExecutor() ).
  m1 = MagenticOne(client=client)
/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:167: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._m

---------- TextMessage (user) ----------
마이크로소프트 현재 주가와 전망에 대해 알려줘
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

마이크로소프트 현재 주가와 전망에 대해 알려줘


To answer this request we have assembled the following team:

FileSurfer: An agent that can handle local files.
WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.
Coder: A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills.
ComputerTerminal: A computer terminal that performs no other action than running Python scripts (provided to it quoted in ```python co

/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:315: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- TextMessage (WebSurfer) ----------
Web surfing error:

Traceback (most recent call last):
  File "/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py", line 440, in on_messages_stream
    content = await self._generate_reply(cancellation_token=cancellation_token)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py", line 467, in _generate_reply
    await self._lazy_init()
  File "/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py", line 324, in _lazy_init
    browser = await self._playwright.chromium.launch(**launch_args)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-p

/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:981: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


...

/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:454: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

마이크로소프트 현재 주가와 전망에 대해 알려줘


To answer this request we have assembled the following team:

FileSurfer: An agent that can handle local files.
WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.
Coder: A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills.
ComputerTerminal: A computer terminal that performs no other action than running Python scripts (provided to it quoted in ```python code blocks), or sh shell scripts (provided to it quoted in ```sh cod

/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:466: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- TextMessage (MagenticOneOrchestrator) ----------
Please execute the command `playwright install` to install the required browser dependencies.
---------- TextMessage (ComputerTerminal) ----------
No code blocks found in the thread. Please provide at least one markdown-encoded code block to execute (i.e., quoting code in ```python or ```sh code blocks).
---------- TextMessage (MagenticOneOrchestrator) ----------
Please execute the command `playwright install` to install the required browser dependencies.
---------- TextMessage (ComputerTerminal) ----------
No code blocks found in the thread. Please provide at least one markdown-encoded code block to execute (i.e., quoting code in ```python or ```sh code blocks).
---------- TextMessage (MagenticOneOrchestrator) ----------
Execute the following shell command:
```sh
playwright install
```
This will install the missing dependencies for WebSurfer.
---------- TextMessage (ComputerTerminal) ----------
No code blocks found in the thr

/Users/andy/works/ai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:481: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
